# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.

## Setup

In [ ]:
from pathlib import Path
from freqtrade.configuration import Configuration

# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["freqtrade-config.json"])
# Optionally, use existing configuration file
# config = Configuration.from_files(["config.json"])

# Define some constants
config["timeframe"] = "1m"
# Name of the strategy class
config["strategy"] = "BBandsStrategy"
# Location of the data
data_location = Path(config['user_data_dir'], 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC/USDT"

In [ ]:
import vectorbt as vbt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit
import itertools
import talib
import ta

In [ ]:

# config = Configuration.from_files(["freqtrade-config.json"])


In [ ]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "json",
                            # data_format = "hdf5",
                            )

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
candles.head()

## Load and run strategy
* Rerun each time the strategy file is changed

In [ ]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles[candles.date >= pd.Timestamp('2021-04-01',).tz_localize('US/Eastern')], {'pair': pair})
df.tail()

### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [ ]:
# Report results
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file 
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"

In [ ]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'PKStrategy'
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

In [ ]:
trades

### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["sell_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '1m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2021-06-01':'2021-06-5']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,bin_size=0.01)
fig.show()


In [290]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
import talib
import pandas_ta as ta
from talib.abstract import *
from math import *
import vectorbt as vbt
import json

# from technical.indicators import accumulation_distribution
import technical.indicators as ti
from technical.util import resample_to_interval, resampled_merge

%matplotlib inline

In [263]:
from pathlib import Path
from freqtrade.configuration import Configuration

# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["freqtrade-config.json"])
# Optionally, use existing configuration file
# config = Configuration.from_files(["config.json"])

# Define some constants
config["timeframe"] = "1m"
# Name of the strategy class
config["strategy"] = "PKStrategy"
# Location of the data
data_location = Path(config['user_data_dir'], 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC/USDT"
##############################

from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "json",
                            # data_format = "hdf5",
                            )

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")

Loaded 256430 rows of data for BTC/USDT from freq-user-data/data/binance


In [276]:


##############################
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)

data = candles.set_index('date', drop=False)
data = data[data.date >= pd.Timestamp('2021-06-01').tz_localize('US/Eastern')]
# Generate buy/sell signals using strategy
# df = strategy.analyze_ticker(data, {'pair': pair})
# df.tail()
# print(f"Generated {ddf['buy'].sum()} buy signals")
# data = df.set_index('date', drop=False)
# data.tail()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.

In [318]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

strategy = StrategyResolver.load_strategy(config)
# ddf = df.copy()
# ddf = data.copy()
ddf = resample_to_interval(data, 5)

dlen = len(ddf)
capital = strategy.config['starting_balance']
sl_n2= IntSlider(description="n2", min=0, max=floor(log(dlen, 2.0)), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=floor(dlen/pow(2,9)), step=1, value=5)
def update_sl_w_range(*args):
    wsize = pow(2,sl_n2.value)
    sl_w.max = floor(dlen/wsize)*2 -1
sl_n2.observe(update_sl_w_range, 'value')


slrfr= IntSlider(description="rfr", min=0, max=10000, step=1, value=0)

ddfreq = Dropdown(options=[('1m', 1), ('5m', 5), ('15m', 15), ('1h', 60), ('2h', 120), ('4h', 240), ('8h', 480)],value=1,description='freq:',disabled=False)

# slCoolOff = IntSlider(description="cool", min=0, step=5, max=600, value=30)
# slDowntrendWait = IntSlider(description="dnt", min=0, max=100, value=30)

slStopLoss = FloatSlider(description="sl", min=0, step=0.05, max=15,value=1.92, continuous_update=False)
slTrailingStopLoss = FloatSlider(description="ts", min=0, step=0.2, max=15,value=10, continuous_update=False)
slTakeProfit = FloatSlider(description="tp", min=0, step=0.2, max=15, value=10, continuous_update=False)

# slMa1N = IntSlider(description="ma1n", min=0, step=3, max=400, value=50)
# slMa2N = IntSlider(description="ma2n", min=0, step=5, max=600, value=100)
# slBBTP = IntSlider(description="bbtp", min=3, step=1, max=200, value=30, continuous_update=False)
# slBBUP = FloatSlider(description="bbup", min=0, step=0.05, max=5,value=2, continuous_update=False)
# slBBDN = FloatSlider(description="bbdn", min=0, step=0.05, max=5,value=2, continuous_update=False)
# slBBMACH = FloatSlider(description="bbch", min=0, step=0.05, max=3,value=0.5, continuous_update=False)
rslMACD = IntRangeSlider(value=[12, 26],min=0,max=480,step=1,description='macd',continuous_update=False)

rslTrendPer = IntRangeSlider(description="trend", min=0, step=1, max=150, value=[20,40])
slBuyChannel = FloatSlider(description="bch", min=-5., step=0.05, max=5,value=0.5, continuous_update=False)
slMACDUp = FloatSlider(description="macdup", min=-0., step=0.05, max=2.,value=0.05, continuous_update=False)
# slBuyTr = IntSlider(description="buytr", min=0, step=10, max=250, value=30)


slRSIPer = IntSlider(description="rsip", min=3, step=1, max=250, value=30)
slRSIBuy = IntSlider(description="rsib", min=7, step=1, max=250, value=30)

sui = widgets.VBox([
    widgets.HBox([slTrailingStopLoss, slStopLoss, slTakeProfit, slBuyChannel]), 
    widgets.HBox([rslMACD, rslTrendPer, slRSIPer, slRSIBuy]),
    widgets.HBox([slMACDUp])
])

wui = widgets.HBox([sl_n2, sl_w])

pf = wst = wed = wsz = wdf = None
ddf_trades = ddf_entries = ddf_profits = ddf_losses = ddf_cap = None

idx_ddf = np.arange(dlen)
# , stk, std, stsm, stt , ma1n, ma2n, ma1c, ma2c, trn, mrn, xrr,
def backtest(sl, ts, tp, macd, trp, rsip, rsib, buych, macdup):
    with open(f'./strat-params.json', "w") as f: dvals = {**swdts,  **wwdts}; f.write(json.dumps({k: v.value for k, v in dvals.items()})) ;
        
    global ddf, pf, ddf_trades, ddf_entries, ddf_profits, ddf_losses, ddf_cap
    
    
#     ddf = strategy.analyze_ticker(data[data.date >= pd.Timestamp('2021-05-01',).tz_localize('US/Eastern')], {'pair': pair})
    
# Generate buy/sell signals using strategy

    strategy.stoploss = -sl/100
    strategy.cust_tp_stop = tp
    strategy.cust_ts_stop = ts
    
#     strategy.cust_buy_threshold = buytr
    strategy.cust_trend_period = trp
    strategy.cust_macd_fast = rslMACD.value[0]    
    strategy.cust_macd_slow = rslMACD.value[1]
    strategy.cust_macd_up = macdup
    
    strategy.cust_buy_channel = buych
    strategy.cust_rsi_period = rsip
    strategy.cust_buy_rsi = rsib

    ddf = strategy.populate_indicators(ddf, {'pair': pair})
    ddf = strategy.populate_buy_trend(ddf, {'pair': pair})
    ddf = strategy.populate_sell_trend(ddf, {'pair': pair})
   

    pf = vbt.Portfolio.from_signals(ddf['close'], entries=ddf['buy'], 
                                    exits=ddf.sell,
                                    direction='longonly', init_cash=capital, fees=0.001, slippage=0.0025, freq='5m')

    ddf_trades = pf.trades.records_readable.copy()    
    ddf_trades['cap'] = ddf_trades['PnL'].cumsum() + capital
    
    
    ddf_entries = ddf_trades.set_index("Entry Date")
    ddf_exits = ddf_trades.set_index("Exit Date")
    
    ddf_profits = ddf_exits[ddf_exits.PnL > 0]
    ddf_losses = ddf_exits[ddf_exits.PnL <= 0]
    
    
    slrfr.value += 1
    with open(f'./strategy-results/{datetime.now():strat_%Y-%m-%d}.log', "a") as f: 
            f.write(json.dumps(dict({
                "dt": f'{datetime.now():%Y-%m-%d %HH:%MM:%SS}',
                "period_start": f'{ddf.index[0]}',
                "period_end": f'{ddf.index[-1]}',
                "params": {k: v.value for k, v in {**swdts,  **wwdts}.items()},
                "strategy": "2EMASTH",
#                 "freq": freq,
                "profit": floor(100* (ddf_trades.iloc[-1].cap / capital - 1)) if len(ddf_trades.index) else 0.
            })
        )); f.write("\n");
                    

    
wddf_entries = wddf_profits = wddf_losses = wddf_trades = None

def showplt(rfr, w2log, w, tp):
    global wst, wed, wsz, wdf, ddf, ddf_entries, ddf_profits, ddf_losses, ddf_cap
    global wddf_entries, wddf_profits, wddf_losses, wddf_trades

        
    if not pf: return
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    
    wdf = ddf[200:].iloc[wst:wed,:]
    
    fig = plt.figure(constrained_layout=True,figsize=(16,5))
    gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

    gs00 = gs0[0,0].subgridspec(4, 1, height_ratios=[4,1,1,1])
    gs01 = gs0[0,1].subgridspec(2, 1)

    ax1 = fig.add_subplot(gs00[0,0])
    ax2 = fig.add_subplot(gs00[1,0])
    ax5 = fig.add_subplot(gs00[2,0])

    ax3 = fig.add_subplot(gs01[0,0])
    ax4 = fig.add_subplot(gs01[1,0])


#     wma1 = wdf['ma1']; wma2 = ma2[wdf.index]; wma3 = ma3[wdf.index];
    
    ax1.plot(wdf['close'], lw=0.5)
    ax1.plot(wdf.macd_fast, lw=0.5)
    ax1.plot(wdf.macd_slow, lw=0.5)
#     ax1.plot(wdf.bb_upperband, lw=0.5)
    # ax1.plot(wdf['rmin'], lw=0.5)
    # ax1.plot(wdf['rmax'], lw=0.5)
    # ax1.plot(wdf['ma1'], lw=0.5)
    # ax1.plot(wdf['ma2'], lw=0.5)
    
#     ax1.fill_between(wdf.index, wdf.bb_ma_lowerband, wdf.bb_ma, alpha=0.15)
    # ax1.fill_between(wdf.index, wdf['ma1'], wdf['ma1c'][wdf.index], alpha=0.15)
    # ax1.fill_between(wdf.index, wdf['ma2'], wdf['ma2c'][wdf.index], alpha=0.15)

    if ddf_trades is not None:
        wddf_entries = ddf_entries[ddf_entries.index.isin(wdf.index)]
        wddf_profits = ddf_profits[ddf_profits.index.isin(wdf.index)]
        wddf_losses = ddf_losses[ddf_losses.index.isin(wdf.index)]

        wddf_trades = ddf_trades[ddf_trades.index.isin(wdf.index)]    

        ax1.vlines(wddf_entries.index,wddf_entries['Avg. Entry Price'] * 0.97,wddf_entries['Avg. Entry Price'] * 1.02, lw=0.5, color='blue')
        ax1.vlines(wddf_profits.index,wddf_profits['Avg. Exit Price'] *0.97,wddf_profits['Avg. Exit Price'] * 1.02, lw=0.5, color='green')
        ax1.vlines(wddf_losses.index,wddf_losses['Avg. Exit Price'] * 0.97,wddf_losses['Avg. Exit Price'] * 1.02, lw=0.5, color='red')
        
        ax1.plot(wddf_entries.index,wddf_entries['Avg. Entry Price']*(1+tp/100), 'o', color='black');
    
#     ax2.plot(wdf.htsine, color="blue")
#     ax2.plot(wdf.htleadsine, color="orange")
    # ax2.plot(wdf.xrr,lw=0.5,color='green')

#     ax2.plot(wslowk,lw=0.5,color='orange')
#     ax2.plot(wslowd,lw=0.5,color='teal')
#     ax2.axhline(y=20, color='b', linestyle='--',lw=0.5)
#     ax2.axhline(y=80, color='b', linestyle='--',lw=0.5)

#     ax2.vlines(wddf_entries.index,0,100, lw=0.5, color='blue')
#     ax2.vlines(wddf_profits.index,0,100, lw=0.5, color='green')
#     ax2.vlines(wddf_losses.index,0,100, lw=0.5, color='red')
#     ax2.hlines(0, 0, len(wdf))
    ax2.plot(wdf['rsi'],lw=0.5)
#     ax2.plot(wdf['macd'],lw=0.5)
#     ax2.plot(wdf['macd_signal'],lw=0.5)
    ax2.axhline(50, lw=0.5, color='0')
    ax2.axhline(30, lw=0.5, color='0')
    ax2.axhline(70, lw=0.5, color='0')

    if ddf_trades is not None:
        ax3.plot(ddf_trades['cap'])
        ax4.plot(wddf_trades['cap'])
    
#     ax5.plot(wdf['rsi'])
    plt.show()
    print(pf.stats())
    print(pf.returns_stats())


swdts = { "sl":slStopLoss, "ts": slTrailingStopLoss, "tp": slTakeProfit, "macd": rslMACD, "trp": rslTrendPer, "rsip": slRSIPer, 
        "rsib": slRSIBuy, "buych": slBuyChannel, "macdup": slMACDUp }

wwdts = {"w2log":sl_n2, "w":sl_w, "tp": slTakeProfit}
dvals = {**swdts,  **wwdts};
with open('./strat-params.json') as f: 
    js = json.loads(f.read());
    for k, v in dvals.items(): 
        if k in js: v.value=js[k] ;

sout = widgets.interactive_output(backtest, swdts)
wout = widgets.interactive_output(showplt, {**wwdts, "rfr": slrfr})
        
display(sui, sout)
display(wui, wout)

@debug_view.capture(clear_output=True)
def sout__(v):
    print(v)
    

Output()

Output()

In [225]:
ddf_entries

,Trade Id,Column,Size,Avg. Entry Price,Entry Fees,Exit Date,Avg. Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id,cap
Entry Date,,,,,,,,,,,,,,
2021-06-01 05:07:00+00:00,0,0,0.269998,37000.340175,9.990010,2021-06-01 13:30:00+00:00,36955.529625,9.977911,-32.066671,-0.003210,Long,Closed,0,9967.933329
2021-06-01 14:43:00+00:00,1,0,0.269242,36985.282625,9.957975,2021-06-04 01:16:00+00:00,37715.056050,10.154461,176.372903,0.017712,Long,Closed,1,10144.306232
2021-06-04 01:29:00+00:00,2,0,0.267028,37951.692625,10.134172,2021-06-04 05:14:00+00:00,36881.205900,9.848322,-305.832634,-0.030178,Long,Closed,2,9838.473598
2021-06-04 05:28:00+00:00,3,0,0.264193,37202.454200,9.828645,2021-06-04 12:09:00+00:00,35750.210475,9.444972,-402.946906,-0.040997,Long,Closed,3,9435.526692
2021-06-04 12:20:00+00:00,4,0,0.261620,36029.709650,9.426101,2021-06-04 12:47:00+00:00,36844.657500,9.639307,194.141428,0.020596,Long,Closed,4,9629.668121
2021-06-04 13:39:00+00:00,5,0,0.260043,36994.114650,9.620048,2021-06-05 09:50:00+00:00,36670.035150,9.535774,-103.430318,-0.010752,Long,Closed,5,9526.237803
2021-06-05 10:08:00+00:00,6,0,0.261012,36460.925000,9.516721,2021-06-05 21:34:00+00:00,34896.190875,9.108307,-427.038645,-0.044872,Long,Closed,6,9099.199158
2021-06-05 21:46:00+00:00,7,0,0.259296,35056.943800,9.090109,2021-06-07 20:48:00+00:00,34605.918375,8.973160,-135.012149,-0.014853,Long,Closed,7,8964.187009
2021-06-07 21:04:00+00:00,8,0,0.258093,34697.738025,8.955232,2021-06-07 21:15:00+00:00,33516.000000,8.650234,-322.603385,-0.036024,Long,Closed,8,8641.583624


In [113]:
# with open(f'./strat-params.json', "w") as f:  f.write(json.dumps({k: v.value for k, v in dvals.items()})) ;


    
    
#     slStopLoss.value=js["sl"]; slTrailingStopLoss.value=js["ts"]; slTakeProfit.value=js["tp"];
#     slBBTP.value=js['bbtp']; slBBUP.value=js['bbup']; slBBDN.value=js['bbdn']; #slBBMACH.value=js["bbch"]; 



IndentationError: unexpected indent (<ipython-input-113-68590dd77297>, line 6)

In [166]:
ddf['bb_lowerband'] - ddf['bb_lowerband'].vbt.fshift(1)

date
2021-06-01 04:00:00+00:00          NaN
2021-06-01 04:01:00+00:00          NaN
2021-06-01 04:02:00+00:00          NaN
2021-06-01 04:03:00+00:00          NaN
2021-06-01 04:04:00+00:00          NaN
                               ...    
2021-06-22 23:53:00+00:00    11.546935
2021-06-22 23:54:00+00:00     0.129647
2021-06-22 23:55:00+00:00     1.083793
2021-06-22 23:56:00+00:00    -0.316016
2021-06-22 23:57:00+00:00     5.095725
Freq: T, Name: bb_lowerband, Length: 31438, dtype: float64

In [285]:
dir(pf)
pf.config

{'wrapper': <vectorbt.base.array_wrapper.ArrayWrapper at 0x7fd212c9bbe0>,
 'close': date
 2021-06-01 04:00:00+00:00    36623.20
 2021-06-01 04:01:00+00:00    36576.52
 2021-06-01 04:02:00+00:00    36685.97
 2021-06-01 04:03:00+00:00    36777.45
 2021-06-01 04:04:00+00:00    36771.52
                                ...   
 2021-06-28 01:45:00+00:00    34428.56
 2021-06-28 01:46:00+00:00    34455.18
 2021-06-28 01:47:00+00:00    34450.87
 2021-06-28 01:48:00+00:00    34467.00
 2021-06-28 01:49:00+00:00    34444.80
 Freq: T, Length: 38750, dtype: float64,
 'order_records': array([( 0,   144, 0, 0.26905746, 37129.65265 , 9.99000999, 0),
        ( 1,   251, 0, 0.26905746, 36118.886475, 9.71805581, 1),
        ( 2,  5649, 0, 0.25756157, 37655.614275, 9.69863912, 0),
        ( 3,  6106, 0, 0.25756157, 37003.788675, 9.53075389, 1),
        ( 4, 11138, 0, 0.28513017, 33359.19    , 9.51171143, 0),
        ( 5, 11338, 0, 0.28513017, 32783.73525 , 9.34763192, 1),
        ( 6, 11762, 0, 0.27381035,

In [194]:
# macd_ind = vbt.MACD.run(
#             ddf['close'],
#             fast_window=12,
#             slow_window=26,
#             signal_window=7
#         )
# dir(macd_ind)
help(macd_ind)

Help on MACD in module vectorbt.indicators.basic object:

class MACD(vectorbt.indicators.factory.IndicatorBase, ParamIndexer)
 |  MACD(wrapper: vectorbt.base.array_wrapper.ArrayWrapper, input_list: List[numpy.ndarray], input_mapper: Union[numpy.ndarray, NoneType], in_output_list: List[numpy.ndarray], output_list: List[numpy.ndarray], param_list: List[List[Any]], mapper_list: List[pandas.core.indexes.base.Index], short_name: str, level_names: Tuple[str, ...]) -> None
 |  
 |  Moving Average Convergence Divergence (MACD).
 |  
 |  Is a trend-following momentum indicator that shows the relationship between
 |  two moving averages of prices.
 |  
 |  See [Moving Average Convergence Divergence – MACD](https://www.investopedia.com/terms/m/macd.asp).
 |  
 |  Method resolution order:
 |      MACD
 |      vectorbt.indicators.factory.IndicatorBase
 |      vectorbt.base.array_wrapper.Wrapping
 |      vectorbt.utils.config.Configured
 |      vectorbt.utils.config.Pickleable
 |      vectorbt.base.

In [ ]:
# bb = vbt.talib('BBANDS').run(ddf.close, timeperiod=2, nbdevup=2, nbdevdn=2)
close_above_bblow = ddf.bb_lowerband > ddf.close
sig = close_above_bblow & ~(close_above_bblow.shift(1, fill_value=False))
sig[sig]

In [ ]:
# dir(bb)
# ddf['bb_lowerband'].vbt.above(1)
#  above(ddf.close, crossover=True)
len(ddf)

In [ ]:
ddf.buy.vbt.signals.first()